In [ ]:
import math
import requests

# --- Haversine Formula ---
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) ** 2 +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(dlon / 2) ** 2)
    c = 2 * math.asin(math.sqrt(a))
    return R * c

# --- Sellers Data ---
sellers = []

def add_seller(shop_name, lat, lon, medicines):
    sellers.append({
        'shop_name': shop_name,
        'lat': lat,
        'lon': lon,
        'medicines': medicines
    })

# --- Add sellers in major cities ---
add_seller("Green Pharmacy - Delhi", 28.6139, 77.2090, {"Paracetamol": 20, "Aspirin": 30})
add_seller("HealthPlus - Mumbai", 19.0760, 72.8777, {"Paracetamol": 19, "Ibuprofen": 25})
add_seller("MediCare - Bangalore", 12.9716, 77.5946, {"Aspirin": 32, "Ibuprofen": 27})
add_seller("Apollo Health - Chennai", 13.0827, 80.2707, {"Paracetamol": 21, "Cetrizine": 15})
add_seller("CityMeds - Hyderabad", 17.3850, 78.4867, {"Cough Syrup": 40, "Paracetamol": 20})
add_seller("Pharma World - Kolkata", 22.5726, 88.3639, {"Ibuprofen": 22, "Paracetamol": 24})
add_seller("Wellness Store - Pune", 18.5204, 73.8567, {"Aspirin": 28, "Cough Syrup": 38})
add_seller("Ahmedabad Medico", 23.0225, 72.5714, {"Paracetamol": 18, "Ibuprofen": 24})
add_seller("Jaipur Pharma", 26.9124, 75.7873, {"Cetrizine": 14, "Aspirin": 29})
add_seller("Lucknow Drug House", 26.8467, 80.9462, {"Paracetamol": 22, "Cough Syrup": 37})

# --- Geocode Address ---
def geocode_address(address):
    url = "https://nominatim.openstreetmap.org/search"
    params = {'q': address, 'format': 'json'}
    headers = {'User-Agent': 'MedicineFinderApp/1.0 (your_email@example.com)'}
    response = requests.get(url, params=params, headers=headers)
    data = response.json()
    if data:
        return float(data[0]['lat']), float(data[0]['lon'])
    else:
        return None, None

# --- Search Nearby Shops ---
def find_medicine_nearby(user_lat, user_lon, medicine_name, max_distance_km=10):
    found_shops = []
    for seller in sellers:
        if medicine_name in seller['medicines']:
            distance = haversine(user_lat, user_lon, seller['lat'], seller['lon'])
            if distance <= max_distance_km:
                found_shops.append({
                    "shop_name": seller['shop_name'],
                    "distance_km": round(distance, 2),
                    "price": seller['medicines'][medicine_name]
                })
    return sorted(found_shops, key=lambda x: x['distance_km'])

# --- Main Program ---
if __name__ == "__main__":
    print("=== 🩺 Nearby Medicine Finder ===")
    #med = input("Enter medicine name: ").strip()

    #print("\nTip: You can just enter your area and city (e.g., 'JP Nagar, Bangalore')")
    #address = input("Enter your area/locality and city: ").strip()

    #user_lat, user_lon = geocode_address(address)

    #if user_lat is None:
       # print("❌ Could not locate your address. Please try again with more detail.")
    #else:
        #results = find_medicine_nearby(user_lat, user_lon, med)

        #if results:
            #print(f"\n✅ Shops near you with '{med}':")
            #for shop in results:
                #print(f" - {shop['shop_name']} ({shop['distance_km']} km) - ₹{shop['price']}")
        #else:
            #print(f"\nNo shops found within 10 km that have '{med}'.")


import ipywidgets as widgets
from IPython.display import display, clear_output
from geopy.geocoders import Nominatim
import math

# ---------------- Data Structures ----------------
users = {}
user_history = {}
current_user = None

sellers = []

def add_seller(shop_name, lat, lon, medicines):
    sellers.append({
        'shop_name': shop_name,
        'lat': lat,
        'lon': lon,
        'medicines': medicines
    })

# Sample sellers
add_seller("Green Pharmacy - Delhi", 28.6139, 77.2090, {"Paracetamol": 20, "Aspirin": 30})
add_seller("HealthPlus - Mumbai", 19.0760, 72.8777, {"Paracetamol": 19, "Ibuprofen": 25})
add_seller("MediCare - Bangalore", 12.9716, 77.5946, {"Aspirin": 32, "Ibuprofen": 27})
add_seller("Apollo Health - Chennai", 13.0827, 80.2707, {"Paracetamol": 21, "Cetrizine": 15})

# ---------------- Haversine ----------------
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat, dlon = math.radians(lat2 - lat1), math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(lat1)) * \
        math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2
    return R * 2 * math.asin(math.sqrt(a))

# ---------------- Geocoder ----------------
def geocode_address(address):
    try:
        geolocator = Nominatim(user_agent="colab_med_finder")
        location = geolocator.geocode(address)
        return (location.latitude, location.longitude) if location else (None, None)
    except Exception:
        return (None, None)

# ---------------- GUI Components ----------------
login_output = widgets.Output()
main_output = widgets.Output()
history_output = widgets.Output()

# Login widgets
username_input = widgets.Text(description="Username:")
password_input = widgets.Password(description="Password:")
login_button = widgets.Button(description="Sign In", button_style='success')
signup_button = widgets.Button(description="Sign Up", button_style='info')

# Search widgets
medicine_input = widgets.Text(description="Medicine:")
address_input = widgets.Text(description="Address:")
search_button = widgets.Button(description="Search Medicine", button_style='primary')
search_results = widgets.Output()

# ---------------- Functions ----------------
def show_login():
    clear_output()
    display(widgets.VBox([username_input, password_input, widgets.HBox([login_button, signup_button]), login_output]))

def show_main_ui():
    clear_output()
    with main_output:
        clear_output()
        display(widgets.HTML(f"<h3>Welcome, {current_user}</h3>"))
        display(widgets.VBox([
            medicine_input,
            address_input,
            search_button,
            search_results,
            widgets.HTML("<h4>Search History:</h4>"),
            history_output
        ]))
    display(main_output)

def update_history():
    history_output.clear_output()
    with history_output:
        history = user_history.get(current_user, [])
        for item in history:
            print("🔍", item)

def on_signin_clicked(_):
    global current_user
    user = username_input.value.strip()
    pwd = password_input.value.strip()
    if user in users and users[user] == pwd:
        current_user = user
        login_output.clear_output()
        show_main_ui()
    else:
        with login_output:
            clear_output()
            print("❌ Incorrect credentials!")

def on_signup_clicked(_):
    user = username_input.value.strip()
    pwd = password_input.value.strip()
    if user in users:
        with login_output:
            clear_output()
            print("❌ Username already exists.")
    else:
        users[user] = pwd
        user_history[user] = []
        with login_output:
            clear_output()
            print("✅ Account created successfully.")

def on_search_clicked(_):
    search_results.clear_output()
    med = medicine_input.value.strip()
    address = address_input.value.strip()

    lat, lon = geocode_address(address)
    if lat is None:
        with search_results:
            print("❌ Couldn't fetch location. Try a more specific address.")
        return

    found = []
    for shop in sellers:
        if med in shop["medicines"] or med.capitalize() in shop["medicines"]:
            dist = haversine(lat, lon, shop["lat"], shop["lon"])
            if dist <= 10:
                found.append(f"✔ {shop['shop_name']} — ₹{shop['medicines'][med.capitalize()]} ({round(dist,2)} km away)")

    user_history[current_user].append(f"{med} @ {address}")
    update_history()

    with search_results:
        if found:
            for shop in found:
                print(shop)
        else:
            print("No shop found nearby with that medicine.")

# ---------------- Callbacks ----------------
login_button.on_click(on_signin_clicked)
signup_button.on_click(on_signup_clicked)
search_button.on_click(on_search_clicked)

# ---------------- Launch App ----------------
show_login()




def on_search_clicked(_):
    ...
    lat, lon = geocode_address(address)
    ...
    for shop in sellers:
        if med in shop["medicines"] or med.capitalize() in shop["medicines"]:
            dist = haversine(lat, lon, shop["lat"], shop["lon"])
            if dist <= 10:
                found.append(f"✔ {shop['shop_name']} — ₹{shop['medicines'][med]} ({round(dist,2)} km away)")

